# Побудова CNN за допомогою Keras, що використовує VGG16 в якості згорткової основи

In [103]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import fashion_mnist
from keras import models
from keras import layers
from keras import optimizers
from keras import utils

In [104]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [105]:
x_train = np.reshape(x_train, (-1, 28, 28, 1))
x_test = np.reshape(x_test, (-1, 28, 28, 1))

x_train = tf.keras.layers.Concatenate()([x_train, x_train, x_train])
x_test = tf.keras.layers.Concatenate()([x_test, x_test, x_test])

y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

print(f"x_train: {x_train.shape}, x_test: {x_test.shape}\ny_train: {y_train.shape}, y_test: {y_test.shape}")

x_train: (60000, 28, 28, 3), x_test: (10000, 28, 28, 3)
y_train: (60000, 10), y_test: (10000, 10)


In [106]:
x_train_res = tf.image.resize(x_train, (32, 32))
x_test_res = tf.image.resize(x_test, (32, 32))

print(f"x_train_res: {x_train_res.shape}, x_test_res: {x_test_res.shape}")

x_train_res: (60000, 32, 32, 3), x_test_res: (10000, 32, 32, 3)


In [107]:
from keras.preprocessing.image import img_to_array


train_X = np.asarray([img_to_array(img) for img in x_train_res])
test_X = np.asarray([img_to_array(img) for img in x_test_res])

In [108]:
from sklearn.model_selection import train_test_split


x_train, x_val, y_train, y_val = train_test_split(train_X,
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=13)

In [109]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")

test_datagen = ImageDataGenerator(rescale=1./255)

In [110]:
train_datagen.fit(x_train)
test_datagen.fit(x_val)

In [111]:
train_generator = train_datagen.flow(
    x_train,
    y_train,
    batch_size=200)

test_generator = test_datagen.flow(
    x_val,
    y_val,
    batch_size=200)

In [112]:
from keras.applications.vgg16 import VGG16


conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
conv_base.trainable = False

original_dim = (32, 32, 3)
target_size = (150, 150)

In [113]:
model = models.Sequential([
    layers.Input(original_dim),
    layers.Lambda(lambda image: tf.image.resize(image, target_size)),
    conv_base,

    layers.Flatten(),
    layers.Dropout(0.3),

    layers.Dense(256, activation="relu", kernel_initializer="he_uniform"),
    layers.Dense(10, activation="softmax"),
])

In [114]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_7 (Lambda)           (None, 150, 150, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_10 (Flatten)        (None, 8192)              0         
                                                                 
 dropout_10 (Dropout)        (None, 8192)              0         
                                                                 
 dense_20 (Dense)            (None, 256)               2097408   
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                                 
 dense_22 (Dense)            (None, 10)              

In [115]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
    metrics=["accuracy"])

In [ ]:
history = model.fit(train_generator,
                    epochs=10,
                    steps_per_epoch=100,
                    validation_data=test_generator,
                    validation_steps=50,
                    shuffle=True)

In [ ]:
epochs = range(1, len(history.history["accuracy"]) + 1)

plt.figure(figsize=(8, 4))
plt.plot(epochs, history.history["accuracy"], ":", label="Training accuracy")
plt.plot(epochs, history.history["val_accuracy"], label="Validation accuracy")

plt.title("Training and validation accuracies")
plt.legend()


plt.figure(figsize=(8, 4))
plt.plot(epochs, history.history["loss"], ":", label="Training loss")
plt.plot(epochs, history.history["val_loss"], label="Validation loss")

plt.title("Training and validation losses")
plt.legend()

plt.show()

In [ ]:
model.save("/content/drive/MyDrive/prepared_train_data/models/vgg16_basesd_model.hdf5")

## Донавчання

In [ ]:
conv_base.summary()

In [ ]:
# Розморозка шарів

# conv_base.trainable = True
# set_trainable = False
# for layer in conv_base.layers:
#     if layer.name == "block5_conv1": # останні шари
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False